#                                          SAT SOLVER

DPLL(Davis–Putnam–Logemann–Loveland) Algorithm is used in this SAT solver.<br>
->It is based on Backtracking for finding the satisfiability of any CNF.<br>
->We have used following rules in implementing DPLL algortihm:<br>
&nbsp; &nbsp; &nbsp; &nbsp;->Unit Propogation<br>
&nbsp; &nbsp; &nbsp; &nbsp;->Boolean Constraint Propogation<br>
&nbsp; &nbsp; &nbsp; &nbsp;->Jeroslow Wang Heuristic <br>


In [528]:
# import necessary module
import time

Start time of the sat solver! time.clock() will return the processor time.

In [529]:
start_time=time.clock()

### reading_ip()<br>
->This function reading_ip() will take filename as input and will split the file line by line.<br>
->Each clause will be stored in a list called as cnf_list.<br>
->This function will return cnf_list, number of variables & number of clauses.

In [530]:
def reading_ip(filename):
    fp = open(filename,"r")
    line = fp.read()
    lines = line.split("\n")
    lines = lines[:-1]
   # print(lines)
    cnf_list = list()
    i=0
    for each_line in lines:
        clause=list()
        if each_line[0]=='c':
            continue
        if each_line[0]=='p':
            n_vars = int(line.split()[2])
            n_clause = int(line.split()[3])
            continue
        for c in each_line[:-2].split():
            x=int(c)
            clause.append(x)
        cnf_list.append(clause)
    return cnf_list, n_vars, n_clause

#####  BackTracking Function
This function will call itself recursively until we get a solution.<br>
Jeroslow Wang Method will return the literal that we will choose to assign a truth value.<br>



In [531]:
def backtracking(c_n_f, Assign_val):
    c_n_f,unit_Assign_val=u_prop(c_n_f) #.............Calling unit propogation function
    Assign_val=Assign_val+unit_Assign_val 
    if c_n_f==-1:
        return []
    if check(c_n_f)==1:
        return Assign_val
    var=heuristic_fun(c_n_f)    #.....................Variable which has to be considered to assign a truth value
    result_ret=[]
    result_ret=bin_const_prop(c_n_f,var)
    Assign_val=Assign_val+[var]
    sol=backtracking(result_ret,Assign_val)
    if check(sol)==1:   #.............................Checking if solution list is empty
        result_ret=[]
        result_ret=bin_const_prop(c_n_f,-var)
        Assign_val=Assign_val+[-var]
        sol=backtracking(result_ret,Assign_val)
    return sol

In [532]:
def check(list1):
    if len(list1)==0:
        return 1
    else:
        return 0

In [533]:
def el_check(a,list1):
    if a in list1:
        return 1
    else:
        return 0

##### Boolean Constraint Propogation
If one of a clause's literal is assigned false and remaining literal is unassigned then assign it true.<br><br>
If all of a clause's literals are assigned false then return UNSATISFIABLE

In [534]:
def bol_const_prop(c_n_f, u):
    new_cnf=[]
    #print(-unit)
    for cl in c_n_f: #............................for every clause in the CNF
        new_clause = list()
        if el_check(u,cl)==1:   #...........................If there's a positive literal in clause then we will continue
            continue
        if el_check(-u,cl)==1:  # .............If there's a negation of that literal in clause then remove all such literals
            for c in cl: #...........................for every literal in clause
                if c!=-u: #..........................check if it is not negated one
                    new_clause.append(c) #...........Add those literals to a new clause list
            if check(new_clause)==1: #...............If new_clause is empty return -1 and this means CNF is UNSAT
                return -1
            new_cnf.append(new_clause) #.............Add the new clause to a new_cnf i.e., modify the cnf
        else:
            new_cnf.append(cl) #.....................Append the remaining clauses to the modified CNF
    return new_cnf

##### Unit Propogation
Checking if there's a single literal clause (unit clause) in CNF.<br>
Then we are calling bol_const_prop() function on those unit clauses<br>.

Reference-https://ocw.mit.edu/courses/aeronautics-and-astronautics/16-412j-cognitive-robotics-spring-2016/readings/MIT16_412JS16_Readings2P3.pdf

In [535]:
def u_prop(c_n_f):
    unit_clause=list()
    for l in c_n_f:
        length=len(l)
        if length==1: #...................................checking if it is a unit clause
            #print(l,c_n_f.index(l))
            unit_clause.append(l) #.........................adding it to the list unit_clause
    #print(unit_clause)
    Assign_val = []
    while unit_clause: #.....................................while there are unit clauses 
        u=unit_clause[0]
        #print(u)
        Assign_val.append(u[0])
       # print(u[0])
        c_n_f=bol_const_prop(c_n_f, u[0]) #.............This will contain the new CNF returned by BCP function
        if c_n_f==-1: #...............if returned value is -1 means a clause in this cnf is false then return -1
            return -1,[]
        if check(c_n_f)==1:
            return c_n_f, Assign_val
        unit_clause=list()
        for l in c_n_f:
            length=len(l)
            if length==1:
                unit_clause.append(l)
    return c_n_f,Assign_val

Jeroslow Wang Method is used as a decision heuristic here.<br>
It chooses the literal that maximises following-<br><br>
$ JW = \sum \limits_{l∈w,w∈φ}2^{-|w|} $<br><br>
where l is a variable of clause w and |w| is the number of literals in clause w.<br>
We have to compute JW for each variable of clause

In [536]:
#We are defining the function heuristic_fun.This heiurisctic will find the varibale with the highest score
def heuristic_fun(c_n_f):
    count={}#.............................defining a dictionary which will store count value of each literal
    for each_clause in c_n_f:
        for l in each_clause:
            if el_check(l,count)==1:
                length=len(each_clause)
                count[l]+=2 ** -length
            else:
                length=len(each_clause)
                count[l]=2 ** -length
    count_list_values=list(count.values())
    count_list_keys=list(count.keys())
    max_key=count_list_keys[count_list_values.index(max(count_list_values))]#finding the literal with maximum score
    return max_key #..............................Return the literal which will have the max score
    

### main()
->This is the main function.<br>-> We need to enter the input file name.<br> ->We are calling the reading_ip fucntion which will do the parsing i.e., read the input file and store the clauuses.<br>->Then we are calling the backtracking function which will return the solution.<br> -> We are writting the output i.e., assignment as well as whether it is satisfiable or not in an output file. 

In [537]:
def main():
    filename='unsat2.cnf'   # Define file name 
    cnf,n_vars,n_clause=reading_ip(filename)  # Parse the file
    #print(cnf)
    sol=list()
    sol=backtracking(cnf,[]) # ....................calling backtracking function and it will return the solution
    
    # write the result in output file and assign the value to variable if satisfiable
    input_file = filename.split(".")[0]
    output_file = input_file+"_output.txt"
    fp = open(output_file,"w")
    if check(sol)==0:               # if solution is present
        for c in range(1,n_vars+1):
            if el_check(c,sol)==0:
                if el_check(-c,sol)==0:
                    sol.append(c)
        sol.sort(key=abs)
        fp.write("SATISFIABLE")
        fp.write("\n")
        fp.write("ASSignment: \n")
        for x in sol:
            x=str(x)
            fp.write(x)
            fp.write(" ")
            #If we want to know whether it is assigned true or false we can use following
        """ if x[0]=='-':
                fp.write(x[1:])
                fp.write(":False  ")
            else:
                fp.write(x[0:])
                fp.write(":True   ")
                """
    else:
        fp.write('UNSATISFIABLE')

Calling Main Function!

In [538]:
main()

Printing the total execution time it takes to decide the satifiability of given cnf!

In [539]:
Ex_time=time.clock()-start_time
print("\nExecution Time->",Ex_time)


Execution Time-> 0.11297000000001844
